# Compare Model vs API Predictions
This notebook loads the trained model directly and compares its prediction with the FastAPI endpoint using the exact same input payload.

In [11]:
# Imports (with fallback install for requests)
import sys, subprocess
def ensure(pkg):
    try:
        __import__(pkg)
    except ImportError:
        print(f'Installing {pkg}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
ensure('requests')
import requests
import joblib
import pandas as pd
import numpy as np
import json
import os

In [12]:
# Configuration: adjust API base URL if needed
API_BASE = 'http://127.0.0.1:8000'  # or 'http://localhost:8000'
API_PREDICT = f'{API_BASE}/api/v1/predict'

# Try to load the SAME model file the API uses to avoid version drift
CANDIDATE_MODEL_PATHS = [
    os.path.join('backend', 'heart_rf_pipeline.pkl'),
    os.path.join('model_artifacts', 'heart_rf_pipeline.pkl'),
    os.path.join('notebook', 'heart_rf_pipeline.pkl'),
    'heart_rf_pipeline.pkl',
 ]
model_path = None
model = None
for p in CANDIDATE_MODEL_PATHS:
    if os.path.exists(p):
        try:
            model = joblib.load(p)
            model_path = p
            break
        except Exception as e:
            print(f'Failed to load {p}: {e}')
if model is None:
    raise FileNotFoundError('Could not find a loadable model in expected locations.')
print({'loaded_model_path': model_path, 'pipeline_steps': list(model.named_steps.keys())})
# Columns the preprocessor expects BEFORE transformation
expected_cols = model.named_steps['preprocessor'].feature_names_in_.tolist()
print({'expected_columns': expected_cols})

{'loaded_model_path': 'heart_rf_pipeline.pkl', 'pipeline_steps': ['preprocessor', 'classifier']}
{'expected_columns': ['gender', 'age', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose', 'pulsePressure']}


In [13]:
# Decision threshold for notebook classifications (match website)
THRESH = 0.30  # Keep in sync with website/API default
print({'notebook_threshold': THRESH})

{'notebook_threshold': 0.3}


In [14]:
# Example input (adjust to match your test case)
sample = {
    'gender': 'Male',            # 'Male'/'Female'
    'age': 58,                   # numeric
    'currentSmoker': 'Yes',      # 'Yes'/'No'
    'cigsPerDay': 12,
    'BPMeds': 'No',
    'prevalentStroke': 'No',
    'prevalentHyp': 'Yes',
    'diabetes': 'No',
    'totChol': 240,
    'sysBP': 138,
    'BMI': 28.5,
    'heartRate': 78,
    'glucose': 95,
    'pulsePressure': 50,         # sysBP - diaBP (you kept this feature)
}

In [15]:
# Direct model inference using the training-time preprocessor
df_in = pd.DataFrame([sample]).copy()
for col in expected_cols:
    if col not in df_in.columns:
        df_in[col] = np.nan
df_in = df_in[expected_cols]
pred_dir = int(model.predict(df_in)[0])
proba_dir = float(model.predict_proba(df_in)[0, 1])
# Apply website threshold for classification parity
pred_dir_thr = int(proba_dir >= THRESH)
print({'mode': 'direct_model', 'prediction': pred_dir, 'prediction_at_thresh': pred_dir_thr, 'probability': proba_dir})

{'mode': 'direct_model', 'prediction': 0, 'prediction_at_thresh': 0, 'probability': 0.14}


In [16]:
# Simulate the API's input conversion (Male/Yes/No -> 1/0) to see its effect on the same pipeline
def convert_yes_no(val):
    if isinstance(val, str):
        return 1 if val.lower() == 'yes' else 0
    return val

def convert_gender(val):
    if isinstance(val, str):
        return 1 if val.lower() == 'male' else 0
    return val

api_like = dict(sample)
api_like['gender'] = convert_gender(api_like.get('gender'))
for k in ['currentSmoker','BPMeds','prevalentStroke','prevalentHyp','diabetes']:
    if k in api_like:
        api_like[k] = convert_yes_no(api_like[k])

df_api_like = pd.DataFrame([api_like])
for col in expected_cols:
    if col not in df_api_like.columns:
        df_api_like[col] = np.nan
df_api_like = df_api_like[expected_cols]

pred_sim = int(model.predict(df_api_like)[0])
proba_sim = float(model.predict_proba(df_api_like)[0, 1])
# Apply website threshold for classification parity
pred_sim_thr = int(proba_sim >= THRESH)
print({'mode': 'simulated_api_conversion', 'prediction': pred_sim, 'prediction_at_thresh': pred_sim_thr, 'probability': proba_sim})

{'mode': 'simulated_api_conversion', 'prediction': 0, 'prediction_at_thresh': 0, 'probability': 0.1}


/Users/vishwaperera/Desktop/Projects/heart-disease/Heart-Disease/myenv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/vishwaperera/Desktop/Projects/heart-disease/Heart-Disease/myenv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [17]:
# Real API call (make sure the FastAPI server is running on API_BASE)
api_payload = dict(sample)  # send raw strings; API will convert internally
resp = None
try:
    resp = requests.post(API_PREDICT, json=api_payload, timeout=10)
    if resp.status_code != 200:
        print({'api_status': resp.status_code, 'detail': resp.text[:300]})
        api_result = None
    else:
        api_result = resp.json()
        print({'api_status': resp.status_code, 'api_result': api_result})
except Exception as e:
    print({'api_error': str(e)})
    api_result = None

{'api_status': 200, 'api_result': {'probability': 0.14, 'prediction': 0, 'threshold': 0.3, 'model_version': 'local-dev'}}


In [18]:
# Compare results
def pct(x):
    return None if x is None else f'{x*100:.2f}%'

summary = {
    'threshold_used': THRESH,
    'direct_model_proba': proba_dir,
    'direct_model_pred': pred_dir,
    'direct_model_pred_at_thresh': int(proba_dir >= THRESH),
    'sim_api_conv_proba': proba_sim,
    'sim_api_conv_pred': pred_sim,
    'sim_api_conv_pred_at_thresh': int(proba_sim >= THRESH),
    'api_proba': (None if api_result is None else api_result.get('probability')),
    'api_pred': (None if api_result is None else api_result.get('prediction')),
    'api_threshold': (None if api_result is None else api_result.get('threshold')),
}
print(json.dumps(summary, indent=2))

# Heuristic explanation
if api_result is not None:
    diff_sim_vs_api = abs(proba_sim - api_result['probability'])
    diff_direct_vs_api = abs(proba_dir - api_result['probability'])
    print({'abs_diff_sim_vs_api': diff_sim_vs_api, 'abs_diff_direct_vs_api': diff_direct_vs_api})
    if diff_direct_vs_api > 1e-6 and diff_sim_vs_api < 1e-4:
        print('\nConclusion: The API converts categorical values (Male/Yes/No) to 1/0 before the pipeline, which changes the encoding compared to direct model input with strings. This explains the probability difference.\n')
    elif diff_direct_vs_api < 1e-6 and (api_result.get('threshold') is None or abs(api_result['threshold'] - THRESH) < 1e-6):
        print('\nConclusion: Results and threshold match; model and API are consistent for this input.\n')
    else:
        print('\nConclusion: Differences remain; check model versions/paths and ensure both sides use the same threshold and preprocessing.\n')
else:
    print('\nAPI result unavailable; ensure the FastAPI server is running at', API_BASE)

{
  "threshold_used": 0.3,
  "direct_model_proba": 0.14,
  "direct_model_pred": 0,
  "direct_model_pred_at_thresh": 0,
  "sim_api_conv_proba": 0.1,
  "sim_api_conv_pred": 0,
  "sim_api_conv_pred_at_thresh": 0,
  "api_proba": 0.14,
  "api_pred": 0,
  "api_threshold": 0.3
}
{'abs_diff_sim_vs_api': 0.04000000000000001, 'abs_diff_direct_vs_api': 0.0}

Conclusion: Results and threshold match; model and API are consistent for this input.



## Example inputs for Low, Medium, and High risk patients

Below are three illustrative input examples you can use to probe the model. They are not medical advice; the actual probability comes from the trained model and can vary with small changes.

Notes
- Categorical fields use the same strings as during training: `Male`/`Female`, `Yes`/`No`.
- The website/notebook classifies using a threshold of 0.30.
- All features shown match the model’s expected raw inputs.

### Low risk (likely below threshold)
```json
{
  "gender": "Female",
  "age": 32,
  "currentSmoker": "No",
  "cigsPerDay": 0,
  "BPMeds": "No",
  "prevalentStroke": "No",
  "prevalentHyp": "No",
  "diabetes": "No",
  "totChol": 175,
  "sysBP": 112,
  "BMI": 22.4,
  "heartRate": 68,
  "glucose": 88,
  "pulsePressure": 38
}
```
Why this is low: young age, normal BP and cholesterol, no smoking, no diabetes/hypertension.

### Medium risk (around the threshold)
```json
{
  "gender": "Male",
  "age": 52,
  "currentSmoker": "Yes",
  "cigsPerDay": 5,
  "BPMeds": "No",
  "prevalentStroke": "No",
  "prevalentHyp": "Yes",
  "diabetes": "No",
  "totChol": 215,
  "sysBP": 135,
  "BMI": 27.8,
  "heartRate": 74,
  "glucose": 102,
  "pulsePressure": 48
}
```
Why this is medium: middle age, light smoking, elevated BP, modestly high cholesterol; may land near ~0.3 depending on the model.

### High risk (likely above threshold)
```json
{
  "gender": "Male",
  "age": 68,
  "currentSmoker": "Yes",
  "cigsPerDay": 20,
  "BPMeds": "Yes",
  "prevalentStroke": "No",
  "prevalentHyp": "Yes",
  "diabetes": "Yes",
  "totChol": 265,
  "sysBP": 168,
  "BMI": 31.5,
  "heartRate": 82,
  "glucose": 165,
  "pulsePressure": 60
}
```
Why this is high: older age, active smoking, hypertension (on meds), diabetes, high systolic BP and cholesterol.

Tip: To test one of these cases in this notebook, replace the `sample = { ... }` cell’s content with one of the JSON objects above and rerun the cells. If your FastAPI server is running, the comparison section will also show API vs model parity.